# Подсчет числа пользовательских сессий
Вам необходимо подсчитать число пользовательских сессий в разбивке по доменам на данных из лог-файла.

**Пользовательская сессия** - это пребывание пользователя на сайте такое, что между двумя последовательными кликами проходит не более 30 минут.
Лог-файл такой же, как и на лекции. Находится в HDFS по пути `/datasets/spark/logsM.txt`

In [1]:
import os
import sys

SPARK_HOME = "/usr/hdp/current/spark2-client"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME

PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.7-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))

import random
SPARK_UI_PORT = random.choice(range(10000, 11000))
print(f"Spark UI port: {SPARK_UI_PORT}")

from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.ui.port", SPARK_UI_PORT)

spark = SparkSession.builder.config(conf=conf).appName("Spark SQL").getOrCreate()

Spark UI port: 10232


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/conda/envs/dsenv/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Задание №1
Создайте `DataFrame` из лог-файла. Схему можно скопировать из лекции.

In [3]:
# Ваш код здесь
from pyspark.sql.types import *

log_schema = StructType(fields=[
    StructField("ip", StringType()),
    StructField("timestamp", LongType()),
    StructField("url", StringType()),
    StructField("size", IntegerType()),
    StructField("code", IntegerType()),
    StructField("ua", StringType())
])

In [4]:
log = spark.read.csv('/datasets/spark/logsM.txt', sep="\t", schema=log_schema)
log

DataFrame[ip: string, timestamp: bigint, url: string, size: int, code: int, ua: string]

In [5]:
log.show()

+---------------+--------------+--------------------+----+----+--------------------+
|             ip|     timestamp|                 url|size|code|                  ua|
+---------------+--------------+--------------------+----+----+--------------------+
|  33.49.147.163|20140101014611|http://news.rambl...| 378| 431|Safari/5.0 (compa...|
| 197.72.248.141|20140101020306|http://news.mail....|1412| 203|Safari/5.0 (compa...|
|  33.49.147.163|20140101023103|http://lenta.ru/4...|1189| 451|Chrome/5.0 (compa...|
|  75.208.40.166|20140101032909|http://newsru.com...|  60| 306|Safari/5.0 (Windo...|
| 197.72.248.141|20140101033626|http://newsru.com...| 736| 307|Chrome/5.0 (compa...|
| 222.131.187.37|20140101033837|http://news.mail....|1017| 416|Opera/5.0 (compat...|
| 197.72.248.141|20140101034726|http://news.rambl...|2042| 428|Safari/5.0 (compa...|
|  33.49.147.163|20140101041149|http://lenta.ru/5...| 444| 203|Chrome/5.0 (compa...|
| 197.72.248.141|20140101050543|http://news.yande...|1197| 500|Ch

## Задание №2
Лог не содержит столбца с доменом. Конечно можно извлечь домен с помощью функции [regexp_extract](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.regexp_extract), но мы так делать не будем. Напишите `pandas_udf`, которая будет извлекать домены из столбца `url`. Результаты применения функции поместите в столбец `domain`.

Для извлечения домена можно воспользоваться функцией [urlparse](https://docs.python.org/3/library/urllib.parse.html#urllib.parse.urlparse)

In [12]:
# Ваш код здесь
import pyspark.sql.functions as f
from urllib.parse import urlparse

def _get_domain(url):
    return urlparse(url).netloc

@f.pandas_udf(StringType())
def get_domain(urls):
    return (urls.apply(_get_domain))

In [13]:
log = log.withColumn('domain', get_domain('url'))

In [14]:
log.show(5)

+--------------+--------------+--------------------+----+----+--------------------+---------------+
|            ip|     timestamp|                 url|size|code|                  ua|         domain|
+--------------+--------------+--------------------+----+----+--------------------+---------------+
| 33.49.147.163|20140101014611|http://news.rambl...| 378| 431|Safari/5.0 (compa...|news.rambler.ru|
|197.72.248.141|20140101020306|http://news.mail....|1412| 203|Safari/5.0 (compa...|   news.mail.ru|
| 33.49.147.163|20140101023103|http://lenta.ru/4...|1189| 451|Chrome/5.0 (compa...|       lenta.ru|
| 75.208.40.166|20140101032909|http://newsru.com...|  60| 306|Safari/5.0 (Windo...|     newsru.com|
|197.72.248.141|20140101033626|http://newsru.com...| 736| 307|Chrome/5.0 (compa...|     newsru.com|
+--------------+--------------+--------------------+----+----+--------------------+---------------+
only showing top 5 rows



## Задание №3
Для разминки давайте подсчитаем сколько дней прошло между первым и последним посещением пользователем нашего домена. Будем считать, что интересующий нас домен `news.mail.ru`. В качестве "уникального" идентификатора пользователя договоримся использовать ip-адрес. Использовать оконные функции в данном задании не надо!

### Задание №3.1
Для выполнения задания №3 понадобится делать операции с датами. Заметьте, что в столбце `timestamp` хранится не настоящий timestamp, а число с датой в формате "yyyyMMddHHmmss". Используя функции из `pyspark.sql.functions`, создайте новый столбец `timestamp`, содержащий в себе UNIX timestamp.

При возникновении ошибок, обратите внимание на типы данных. Возможно их нужно привести.

In [ ]:
# Ваш код здесь
# def unix_timestamp(timestamp=None, format='yyyy-MM-dd HH:mm:ss'):


def _to_unix_timestamp(ts):
    return f.unix_timestamp(str(ts), format='yyyyMMddHHmmss')

@f.pandas_udf(StringType())
def to_unix_timestamp(urls):
    return (urls.apply(_get_domain))
f.unix_timestamp()

In [15]:
log = log.filter(log.domain == 'news.mail.ru')
log = log.withColumn('timestamp', f.unix_timestamp(f.col('timestamp').cast('string'), 'yyyyMMddHHmmss'))

In [16]:
log.show(5)

+--------------+----------+--------------------+----+----+--------------------+------------+
|            ip| timestamp|                 url|size|code|                  ua|      domain|
+--------------+----------+--------------------+----+----+--------------------+------------+
|197.72.248.141|1388541786|http://news.mail....|1412| 203|Safari/5.0 (compa...|news.mail.ru|
|222.131.187.37|1388547517|http://news.mail....|1017| 416|Opera/5.0 (compat...|news.mail.ru|
| 75.208.40.166|1388563282|http://news.mail....| 877| 301|Chrome/5.0 (compa...|news.mail.ru|
| 33.49.147.163|1388568276|http://news.mail....| 732| 409|Opera/5.0 (compat...|news.mail.ru|
|110.91.102.196|1388569315|http://news.mail....| 448| 201|Chrome/5.0 (compa...|news.mail.ru|
+--------------+----------+--------------------+----+----+--------------------+------------+
only showing top 5 rows



### Задание №3.2
Приведя timestamp к правильному формату, решите исходную задачу. В результате должен получится `DataFrame` с двумя столбцами `ip` и `days`. Отсортируйте результат по столбцу `days` в порядке убывания и выведите первые 20 строк.

In [ ]:
# Ваш код здесь

## Задание №4
Подсчитайте число сессий, которое каждый пользователь (уникальный ip) сделал на домене `news.mail.ru`. Для решения этой задачи потребуется использование оконных функций (что это такое чуть ниже). Для работы с окнами в Spark SQL используется метод [over()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.over). Само окно определяется с помощью класса [Window](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Window). Резудьтатом будет `DataFrame` со столбцами `ip` и `sessions`, отсортированный в порядке убывания числа сессий.

_Оконная функция_ выполняет вычисления для набора строк, некоторым образом связанных с текущей строкой. Можно сравнить её с агрегатной функцией, но, в отличие от обычной агрегатной функции, при использовании оконной функции несколько строк не группируются в одну, а продолжают существовать отдельно. Внутри же, оконная функция, как и агрегатная, может обращаться не только к текущей строке результата запроса.

![](https://www.sqlitetutorial.net/wp-content/uploads/2018/11/SQLite-window-function-vs-aggregate-function.png)

Вот пример, показывающий, как сравнить зарплату каждого сотрудника со средней зарплатой его отдела:

```sql
SELECT depname, empno, salary, avg(salary) OVER (PARTITION BY depname)
  FROM empsalary;
```

```
  depname  | empno | salary |          avg          
-----------+-------+--------+-----------------------
 develop   |    11 |   5200 | 5020.0000000000000000
 develop   |     7 |   4200 | 5020.0000000000000000
 develop   |     9 |   4500 | 5020.0000000000000000
 develop   |     8 |   6000 | 5020.0000000000000000
 develop   |    10 |   5200 | 5020.0000000000000000
 personnel |     5 |   3500 | 3700.0000000000000000
 personnel |     2 |   3900 | 3700.0000000000000000
 sales     |     3 |   4800 | 4866.6666666666666667
 sales     |     1 |   5000 | 4866.6666666666666667
 sales     |     4 |   4800 | 4866.6666666666666667
(10 rows)
```

[Документация PostgreSQL](https://postgrespro.ru/docs/postgrespro/12/tutorial-window)

In [17]:
# Ваш код здесь
from pyspark.sql import Window

In [18]:
user_window = Window.partitionBy("ip").orderBy("timestamp")

In [22]:
log.select("ip", "timestamp", f.lead("timestamp").over(user_window).alias("lead"))\
    .select("ip", "timestamp", (f.col("timestamp") - f.col("lead")).alias("diff"))\
    .where("diff <= -1800 or diff is NULL")\
    .groupBy("ip").agg(f.count("*").alias("sessions"))\
    .orderBy(f.col("sessions").desc())\
    .show(10)

+---------------+--------+
|             ip|sessions|
+---------------+--------+
|  75.208.40.166|     349|
| 197.72.248.141|     283|
|  33.49.147.163|     246|
| 222.131.187.37|     163|
|135.124.143.193|     149|
| 168.255.93.197|     139|
| 56.167.169.126|     129|
|   49.203.96.67|     117|
|   49.105.15.79|     113|
| 110.91.102.196|      74|
+---------------+--------+
only showing top 10 rows



## Задание №5
Нарисуйте гистограмму распределения числа сессий

In [ ]:
# Ваш код здесь

In [ ]:
spark.stop()